This is the bayesian optimizer with an example

In [51]:
#imports
import numpy as np
from bayes_opt import BayesianOptimization

In [52]:
#function that runs the guesses of the reaction condition optimization algorithm
def get_experiment(T,P,S0,Ccat0): #the function takes the to optimize variables
    'This function is to illustrate the bayesian optimizer'
    R= 8.314 #define needed constants
    KH= 1282
    CH2 = P/KH
    k1 = 5.207*np.exp(-15879/(R*T)) #now if i have found my arrhenius constants and activation energies i put them in like this
    k2f = 0.7366*np.exp(-10143/(R*T))
    k2b = 0.3866*np.exp(-18522/(R*T))
    k3 = 0.000214*np.exp(-21411/(R*T))
    k4 = 2.72*np.exp(-12860/(R*T))
    k5 = 649*np.exp(-37073/(R*T))
    init = [S0,0,0,0,Ccat0,0,0,0] #define how the initial conditon is made from the optimized variables
    def diff(x, init): #the differential system to simulate
        CE, CA1, CA2, CtE,Cprecat, Ccat, Ccatdead, H2uptake = init
        dEdt = -k1*CE*Cprecat - k2f*CE*CA1 + k2b*CtE*CA2
        dA1dt = k1*CE*Cprecat - k2f*CE*CA1 + k2b*CtE*CA2 + 2*k3*CtE*Cprecat
        dA2dt = k1*CE*Cprecat + k2f*CE*CA1 - k2b*CtE*CA2
        dtEdt = k2f*CE*CA1 - k2b*CtE*CA2 - k3*CtE*Cprecat
        dprecatdt = -k1*CE*Cprecat - k3*CtE*Cprecat + k4*Ccat*CH2 - k5*Cprecat
        dcatdt = k1*CE*Cprecat + k3*CtE*Cprecat - k4*Ccat*CH2
        dcatdeaddt = k5*Cprecat
        dH2uptakedt = k4*Ccat*CH2
        return dEdt, dA1dt, dA2dt, dtEdt, dprecatdt, dcatdt,dcatdeaddt, dH2uptakedt

    sol = integrate.solve_ivp(diff, [0,time_1], init, t_eval = timespan)
    

    return (np.average(sol.y[1]) + np.average(sol.y[2]))/(S0*Ccat0) #return what you want to optimize
    #i optimize for total alcohol concentration per substrate per catalyst

In [53]:
# give the bounds in pbounds of all optimized variables
pbounds = {'T': (273,453), 'P': (40,50), 'S0': (1,10), 'Ccat0' :(0.1,1)}

optimizer = BayesianOptimization(
    f=get_experiment,
    pbounds=pbounds,
)

In [54]:
#find the optimum, init points is the amount of random guesses, n_iter is the number of guesses, for kappa see the bayesianoptimization
#github, but 1 should be okay
optimizer.maximize(
    init_points=10,
    n_iter=100,
    kappa = 1.5,
)

|   iter    |  target   |   Ccat0   |     P     |    S0     |     T     |
-------------------------------------------------------------------------
|  1        |  0.5793   |  0.9782   |  49.34    |  2.773    |  328.4    |
|  2        |  0.8797   |  0.9524   |  45.72    |  1.672    |  401.4    |
|  3        |  0.6058   |  0.792    |  43.92    |  2.272    |  310.3    |
|  4        |  0.7989   |  0.1066   |  42.38    |  2.772    |  389.6    |
|  5        |  0.4954   |  0.1696   |  43.12    |  5.659    |  442.0    |
|  6        |  0.9985   |  0.7616   |  48.82    |  1.436    |  386.1    |
|  7        |  0.7      |  0.448    |  47.88    |  2.795    |  353.4    |
|  8        |  0.3113   |  0.9084   |  47.43    |  7.424    |  356.8    |
|  9        |  0.367    |  0.907    |  41.96    |  6.396    |  375.4    |
|  10       |  0.6338   |  0.6282   |  49.82    |  3.37     |  366.8    |
|  11       |  0.4591   |  0.4686   |  41.35    |  5.625    |  413.6    |
|  12       |  0.4304   |  0.256    | 

|  68       |  0.9541   |  0.1      |  46.08    |  2.482    |  425.5    |
|  69       |  1.644    |  0.1      |  43.58    |  1.0      |  427.9    |
|  70       |  1.509    |  0.1718   |  41.13    |  1.082    |  428.8    |
|  71       |  0.8336   |  0.3045   |  42.63    |  2.792    |  429.7    |
|  72       |  1.167    |  0.6875   |  41.02    |  1.01     |  427.4    |
|  73       |  1.011    |  1.0      |  44.47    |  1.0      |  426.9    |
|  74       |  1.257    |  0.3166   |  49.57    |  1.469    |  400.3    |
|  75       |  1.066    |  0.7829   |  46.85    |  1.158    |  393.0    |
|  76       |  0.9627   |  0.9845   |  49.62    |  1.244    |  424.6    |
|  77       |  1.353    |  0.4117   |  43.52    |  1.074    |  395.7    |
|  78       |  1.697    |  0.1      |  50.0     |  1.0      |  388.0    |
|  79       |  1.069    |  0.4734   |  48.62    |  1.777    |  420.1    |
|  80       |  1.158    |  0.1398   |  48.0     |  1.789    |  395.3    |
|  81       |  1.597    |  0.186    | 